# Домашнее задание к уроку 2

Подготовим данные

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [4]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [5]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [18]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
from nltk import ngrams
import nltk
from nltk import collocations 
nltk.download('genesis')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\karas\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)


In [9]:
from collections import Counter

In [10]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2859146


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [12]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69267),
 ('и', 54916),
 ('в', 52853),
 ('я', 52506),
 ('RT', 38070),
 ('на', 35715),
 ('http', 32992),
 ('что', 31472),
 ('...', 28773),
 ('с', 27177)]

-----

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [14]:
# Подготовим три группы токенов: с высокой, средней и низкой частотой

high_tokens = set()
med_tokens = set()
lit_tokens = set()
h = 10000 # если частота выше этого значения будем считать что это токен с высокой частотой
m = 500 # если частота ниже этого значения будем считать что это токен с низкой частотой
for i in freq_dict_sorted:
    if i[1] > h:
        high_tokens.add(i[0])
    elif i[1] < m:
        lit_tokens.add(i[0])
    else:
        med_tokens.add(i[0])
print(len(high_tokens),len(med_tokens),len(lit_tokens))                     
                       

28 499 359062


In [15]:
# создадим фильтр (стопслова) с добавлением токенов средней и низкой частоты, 
# то есть будем считать только по токенам с высокой частотой
sw = noise + list(lit_tokens) + list(med_tokens)

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.80      0.57      0.67     38802
    positive       0.43      0.69      0.53     17907

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.68      0.61      0.62     56709

Wall time: 1min 11s


In [20]:
# посчитаем только по токенам с средней частотой
sw = noise + list(high_tokens) + list(lit_tokens)

In [21]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.56      0.68      0.61     22856
    positive       0.75      0.64      0.69     33853

    accuracy                           0.65     56709
   macro avg       0.65      0.66      0.65     56709
weighted avg       0.67      0.65      0.66     56709

Wall time: 1min 14s


In [22]:
# посчитаем только по токенам с низкой частотой
sw = noise + list(high_tokens) + list(med_tokens)

In [23]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.79      0.72      0.75     30596
    positive       0.70      0.78      0.74     26113

    accuracy                           0.74     56709
   macro avg       0.75      0.75      0.74     56709
weighted avg       0.75      0.74      0.75     56709

Wall time: 53.7 s


### Вывод:

Лучшая точность получилась на токенах с низкой частотностью, конечно есть зависимость от того как смещать границы разделения частотности, но если за границу брать именно количественное значение частотности, то все равно лучший результат дает низкая частотность (по крайней мере в словах)


## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Оставим знаки пунктуации  
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27855
    positive       1.00      1.00      1.00     28854

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [28]:
# похоже что существенный вклад дают симолы которые участвуют в смайлах. Проверим
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [29]:
p = list('"#$%&\'+,.<=>?@[\\]^`{|}~')
p

['"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '+',
 ',',
 '.',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '`',
 '{',
 '|',
 '}',
 '~']

In [30]:
myfilter = stopwords.words('russian') + p
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27866
    positive       1.00      1.00      1.00     28843

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Гипотеза подтверждается

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [31]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32792
    positive       0.83      1.00      0.91     23917

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



In [32]:
pp = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    pp.append((i,accuracy_score(pred, y_test)))
print(pp)

[('!', 0.5151210566224056), ('"', 0.5037824683912606), ('#', 0.5026715336190023), ('$', 0.4916503553227883), ('%', 0.4939603942936747), ('&', 0.4915798197816925), ("'", 0.4914916503553228), ('(', 0.02606288243488688), (')', 0.9132589183374773), ('*', 0.5108360225008376), ('+', 0.4917914264049798), (',', 0.5032005501772205), ('-', 0.5058985346241337), ('.', 0.5058632668535858), ('/', 0.5435645135692747), (':', 0.550529898252482), (';', 0.49484208855737183), ('<', 0.49150928424059676), ('=', 0.49198539914299316), ('>', 0.49150928424059676), ('?', 0.5026009980779065), ('@', 0.5704738224973108), ('[', 0.4913329453878573), ('\\', 0.49150928424059676), (']', 0.4914916503553228), ('^', 0.49796328625085967), ('_', 0.5156853409511718), ('`', 0.49110370487929605), ('{', 0.49147401647004885), ('|', 0.48733005343067237), ('}', 0.4915445520111446), ('~', 0.49150928424059676)]


Выведем фичи с наибольшей значимостью

In [33]:
for i in sorted(pp, key=lambda x: -x[1])[:10]:
    print(i[0],'  -   ' ,i[1])


)   -    0.9132589183374773
@   -    0.5704738224973108
:   -    0.550529898252482
/   -    0.5435645135692747
_   -    0.5156853409511718
!   -    0.5151210566224056
*   -    0.5108360225008376
-   -    0.5058985346241337
.   -    0.5058632668535858
"   -    0.5037824683912606


----



### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [41]:
%%time
myfilter = stopwords.words('russian') + p

count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       1.00      0.55      0.71     50261
    positive       0.22      1.00      0.37      6448

    accuracy                           0.61     56709
   macro avg       0.61      0.78      0.54     56709
weighted avg       0.91      0.61      0.67     56709

Wall time: 53.8 s


In [47]:
%%time
myfilter = stopwords.words('russian')
tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27855
    positive       1.00      1.00      1.00     28854

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709

Wall time: 52.7 s


In [45]:
from sklearn.feature_extraction.text import HashingVectorizer

In [49]:
vectorizer = HashingVectorizer(n_features=2**4,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.51      0.54      0.52     26242
    positive       0.58      0.55      0.56     30467

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.54     56709



In [50]:
vectorizer = HashingVectorizer(n_features=2**8,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.61      0.61      0.61     27624
    positive       0.63      0.62      0.62     29085

    accuracy                           0.62     56709
   macro avg       0.62      0.62      0.62     56709
weighted avg       0.62      0.62      0.62     56709



In [51]:
vectorizer = HashingVectorizer(n_features=2**10,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.64      0.65      0.65     27507
    positive       0.67      0.66      0.66     29202

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.66      0.65      0.66     56709



In [48]:
vectorizer = HashingVectorizer(n_features=2**12,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))


              precision    recall  f1-score   support

    negative       0.68      0.69      0.69     27359
    positive       0.71      0.70      0.70     29350

    accuracy                           0.69     56709
   macro avg       0.69      0.69      0.69     56709
weighted avg       0.69      0.69      0.69     56709

